# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import citrine
from citrine import Citrine

# Testing

In [36]:
client = Citrine(host=os.environ["CITRINE_DEV_HOST"], api_key=os.environ["CITRINE_DEV_API_KEY"])

project = client.projects.get("ce47a13c-d873-452f-ac72-2ee82a29a9fa")
project.name

{'project': {'id': 'ce47a13c-d873-452f-ac72-2ee82a29a9fa', 'name': '[PRELOAD] Seeding Project v33', 'description': '[PRELOAD] Seeding Project v33', 'created_at': '2022-06-30T19:54:34.085Z', 'team': {'id': 'd9687490-8fd9-4ed5-bd67-d4100fbf575d', 'name': '[PRELOAD] Seeding Project v33', 'description': '[PRELOAD] Seeding Project v33', 'created_at': '2022-06-30T19:54:33.980Z'}}}


/Users/sfriedowitz/Documents/citrine/citrine_python/src/citrine/citrine.py:67: UserWarning: Your Citrine Platform deployment has migrated to include the Teams release.See our FAQ for details.
  warn("Your Citrine Platform deployment has migrated to include the Teams release."


'[PRELOAD] Seeding Project v33'

In [55]:
from citrine.informatics.predictors import *

In [56]:
graph = project.predictors.get("5c2f1b8b-b107-4985-8927-51a083dd3121")

In [57]:
ml = graph.predictors[0]
aml.estimators = {"RANDOM_FOREST"}

aml.estimators

{<AutoMLEstimator.RANDOM_FOREST: 'RANDOM_FOREST'>}

In [52]:
graph = project.predictors.update(graph)

None


In [15]:
graph = project.predictors.get(graph.uid)

graph.status_info

https://development.citrine-platform.com/api/v3/projects/ce47a13c-d873-452f-ac72-2ee82a29a9fa/predictors/5c2f1b8b-b107-4985-8927-51a083dd3121


[]

In [77]:
summary = graph.report.model_summaries[0]
summary.model_settings

{'Algorithm': 'Ensemble of non-linear estimators',
 'Number of estimators': 64,
 'Maximum tree depth': 30,
 'Number of features considered at each split': 35,
 'Uses jackknife method of uncertainty estimation': True,
 'Leaf model': 'Mean',
 'Minimum samples per leaf': 1}

In [90]:
summary.selection_summary.evaluation_results[0]["Measurement~Band gap"].metrics["R^2"].mean

0.5433535272138262

In [91]:
summary.selection_summary.evaluation_results[1]["Measurement~Band gap"].metrics["R^2"].mean

0.5132801869612528

In [92]:
summary.selection_summary.evaluation_results[2]["Measurement~Band gap"].metrics["R^2"].mean

0.6993014513534479